# Dashboard for DELFIN CO2

Generate the CONTROL-file for DELFIN CO2 binding studies and submit jobs to bwUniCluster.

In [ ]:
# === DELFIN CO2 Dashboard ===
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import py3Dmol
import os
import subprocess
import shutil
import re
import sys
from pathlib import Path

# Pfade dynamisch ermitteln
HOME = Path.home()

try:
    import IPython
    NOTEBOOK_DIR = Path(IPython.extract_module_locals()[1]['__vsc_ipynb_file__']).parent
except:
    NOTEBOOK_DIR = Path.cwd()

def find_root_dir(start_dir: Path):
    cur = start_dir
    while True:
        if (cur / "software" / "delfin").is_dir():
            return cur
        if cur == cur.parent:
            return None
        cur = cur.parent

ROOT_DIR = find_root_dir(NOTEBOOK_DIR)
if ROOT_DIR is None:
    ROOT_DIR = HOME

BASE_DIR = ROOT_DIR / "calc_co2"

# SUBMIT_TEMPLATES_DIR suchen (nutzt dieselben Templates wie DELFIN)
_submit_candidates = [
    ROOT_DIR / "software" / "delfin" / "examples" / "example_Job_Submission_Scripts" / "BwUniCluster" / "submit_sh",
    NOTEBOOK_DIR / "submit_sh",
]
SUBMIT_TEMPLATES_DIR = None
for _path in _submit_candidates:
    if _path.exists():
        SUBMIT_TEMPLATES_DIR = _path
        break
if SUBMIT_TEMPLATES_DIR is None:
    SUBMIT_TEMPLATES_DIR = _submit_candidates[0]

JOB_TYPE_TEMPLATES = {
    'Short (24h)': 'submit_short.sh',
    'Standard (48h)': 'submit_standard.sh',
    'Long (72h)': 'submit_long.sh',
    'Extra Long (120h)': 'submit_extralong.sh'
}

def patch_submit_for_co2(script_text):
    """Replace 'delfin' with 'delfin co2' and adjust file copy for CO2 module."""
    script_text = re.sub(r'^(\s*)delfin\s*$', r'\1delfin co2', script_text, flags=re.MULTILINE)
    script_text = re.sub(
        r'cp -a "\$SLURM_SUBMIT_DIR"/\{CONTROL\.txt,input\.txt\}',
        'cp -a "$SLURM_SUBMIT_DIR"/{CONTROL.txt,input.xyz,co2.xyz}',
        script_text
    )
    return script_text

# Feste CO2 Koordinaten
CO2_XYZ_CONTENT = """O      0.000000    0.000000    1.840000
C      0.000000    0.000000    3.000000
O      0.000000    0.000000    4.160000
"""

def clean_xyz_input(input_text):
    """Entfernt XYZ-Header falls vorhanden."""
    text = input_text.strip()
    if not text:
        return ''
    
    lines = text.split('\n')
    if len(lines) < 2:
        return text
    
    first_line = lines[0].strip()
    try:
        int(first_line)
        return '\n'.join(lines[2:]).strip()
    except ValueError:
        return text

def parse_resource_settings(control_text):
    pal_match = re.search(r'^\s*PAL\s*=\s*(\d+)', control_text, flags=re.MULTILINE)
    maxcore_match = re.search(r'^\s*maxcore\s*=\s*(\d+)', control_text, flags=re.MULTILINE)
    pal = int(pal_match.group(1)) if pal_match else None
    maxcore = int(maxcore_match.group(1)) if maxcore_match else None
    return pal, maxcore

def apply_resources_to_submit_script(script_text, pal, maxcore):
    if pal:
        script_text = re.sub(r'(?m)^#SBATCH\s+--ntasks=\d+.*$', f'#SBATCH --ntasks={pal}', script_text)
    if pal and maxcore:
        mem_mb = pal * maxcore
        script_text = re.sub(r'(?m)^#SBATCH\s+--mem=\S+.*$', f'#SBATCH --mem={mem_mb}M', script_text)
    return script_text

# CO2 CONTROL.txt Template
DEFAULT_CONTROL_CO2 = """# Input / Output
------------------------------------
xyz=input.xyz
out=complex_aligned.xyz
co2=co2.xyz

# Charge & Multiplicity
------------------------------------
charge=[CHARGE]
multiplicity=[MULTIPLICITY]
broken_sym=

# Solvation
------------------------------------
implicit_solvation_model=CPCM
solvent=[SOLVENT]

# Orientation Scan (single points)
------------------------------------
orientation_distance=4.0
rot_step_deg=10
rot_range_deg=180

# Method Settings
------------------------------------
functional=PBE0
disp_corr=D4
ri_jkx=RIJCOSX
aux_jk=def2/J
main_basisset=def2-SVP
metal_basisset=def2-TZVP
first_coordination_sphere_metal_basisset=no
first_coordination_sphere_scale=1.20
second_coordination_sphere_metal_basisset=no
second_coordination_sphere_scale=1.30
orientation_job=SP
scan_job=OPT

# Relaxed Distance Scan
------------------------------------
scan_end=1.6
scan_steps=25

# Alignment (0-based indices)
------------------------------------
metal=auto
metal_index=
align_bond_index=
neighbors=

# CO2 placement
------------------------------------
place_axis=z
mode=side-on
perp_axis=y
place_optimize=true
place_samples=800
place_clearance_scale=1.0
no_place_co2=false

# Resources
------------------------------------
PAL=12
maxcore=3800

# Parallelization (orientation scan only)
------------------------------------
parallel_orientation_scan=true
max_workers=4

# Alternative keywords (commented examples)
# orientation_job=GFN2-XTB
# scan_job=GFN2-XTB OPT
# broken_sym=%SCF BrokenSym M,N END
#
# Parallelization notes:
# - Each worker uses PAL cores for ORCA
# - max_workers is auto-calculated as: total_cores / PAL
# - Example: 32 cores with PAL=32 -> 1 worker (sequential)
# - Example: 32 cores with PAL=8 -> 4 workers (parallel)
# - Set max_workers explicitly to override (but stay within limits!)
"""

common_layout = widgets.Layout(width='500px')
common_style = {'description_width': 'initial'}

# === Widgets ===
job_name_widget = widgets.Text(
    value='',
    placeholder='z.B. Fe_Complex_CO2',
    description='Job Name:',
    layout=common_layout,
    style=common_style
)

job_type_widget = widgets.RadioButtons(
    options=['Short (24h)', 'Standard (48h)', 'Long (72h)', 'Extra Long (120h)'],
    value='Standard (48h)',
    description='Job Type:',
    style=common_style,
    layout=widgets.Layout(width='500px')
)

coords_help = widgets.Label("Molecule XYZ coordinates (header auto-removed):")
coords_widget = widgets.Textarea(
    value='',
    placeholder='Paste XYZ coordinates of your metal complex:\n\nExample:\n42\nComment line\nFe  0.0  0.0  0.0\nN   1.5  0.0  0.0\n...',
    description='Molecule XYZ:',
    layout=widgets.Layout(width='500px', height='200px'),
    style=common_style
)

# CO2 Koordinaten Anzeige
co2_info = widgets.HTML(
    value=f'<b>CO2 coordinates (auto-generated as co2.xyz):</b><pre style="background:#e8f5e9;padding:8px;border-radius:4px;">{CO2_XYZ_CONTENT}</pre>',
    layout=widgets.Layout(width='500px')
)

control_help = widgets.Label("CONTROL.txt for CO2 binding study - edit as needed:")
control_widget = widgets.Textarea(
    value=DEFAULT_CONTROL_CO2,
    description='CONTROL.txt:',
    layout=widgets.Layout(width='500px', height='400px'),
    style=common_style
)

# Submit Button
submit_button = widgets.Button(
    description='SUBMIT CO2 JOB',
    button_style='success',
    layout=widgets.Layout(width='200px', height='50px')
)

output_area = widgets.Output()

# Molecule visualization
mol_output = widgets.Output(layout=widgets.Layout(
    border='2px solid #4caf50',
    width='600px',
    height='450px',
    overflow='hidden'
))

def update_molecule_view(change=None):
    with mol_output:
        clear_output()
        raw_input = coords_widget.value.strip()
        if raw_input:
            coords = clean_xyz_input(raw_input)
            lines = [l for l in coords.split('\n') if l.strip()]
            num_atoms = len(lines)
            if num_atoms > 0:
                xyz_data = f"{num_atoms}\nMolecule\n{coords}"
                mol_view = py3Dmol.view(width=590, height=440)
                mol_view.addModel(xyz_data, "xyz")
                mol_view.setStyle({}, {"stick": {"radius": 0.15}, "sphere": {"scale": 0.22}})
                mol_view.zoomTo()
                mol_view.show()
            else:
                print("No valid coordinates found.")
        else:
            print("Please enter XYZ coordinates of your molecule.")

coords_widget.observe(update_molecule_view, names='value')

def reset_form():
    job_name_widget.value = ''
    coords_widget.value = ''
    control_widget.value = DEFAULT_CONTROL_CO2
    job_type_widget.value = 'Standard (48h)'
    with mol_output:
        clear_output()
        print("Please enter XYZ coordinates of your molecule.")

def handle_submit(button):
    with output_area:
        clear_output()
        
        job_name = job_name_widget.value.strip()
        job_type = job_type_widget.value
        control_content = control_widget.value
        raw_input = coords_widget.value.strip()
        
        if not job_name:
            print("Error: Job name cannot be empty!")
            return
        
        if not raw_input:
            print("Error: Molecule XYZ coordinates cannot be empty!")
            return
        
        xyz_content = clean_xyz_input(raw_input)
        if not xyz_content:
            print("Error: No valid XYZ coordinates found!")
            return
        
        lines = [l for l in xyz_content.split('\n') if l.strip()]
        if len(lines) < 1:
            print("Error: XYZ coordinates seem empty!")
            return
        
        safe_job_name = "".join(c for c in job_name if c.isalnum() or c in ('_', '-'))
        if not safe_job_name:
            print("Error: Job name contains only invalid characters!")
            return
        
        job_dir = BASE_DIR / safe_job_name
        
        template_filename = JOB_TYPE_TEMPLATES.get(job_type, 'submit_standard.sh')
        template_path = SUBMIT_TEMPLATES_DIR / template_filename
        
        if not template_path.exists():
            print(f"Error: Template not found: {template_path}")
            print(f"Please create the template file for job type '{job_type}'")
            return
        
        try:
            job_dir.mkdir(parents=True, exist_ok=True)
            
            # CONTROL.txt schreiben
            control_path = job_dir / "CONTROL.txt"
            control_path.write_text(control_content)
            
            # input.xyz schreiben (Molekuel XYZ mit Header)
            num_atoms = len(lines)
            xyz_with_header = f"{num_atoms}\n{safe_job_name}\n{xyz_content}\n"
            input_path = job_dir / "input.xyz"
            input_path.write_text(xyz_with_header)
            
            # co2.xyz schreiben
            co2_path = job_dir / "co2.xyz"
            co2_with_header = f"3\nCO2\n{CO2_XYZ_CONTENT}"
            co2_path.write_text(co2_with_header)
            
            # Submit-Script aus bestehendem Template + CO2-Anpassungen
            template_content = template_path.read_text()
            submit_script = template_content.replace('{job_name}', safe_job_name)
            submit_script = patch_submit_for_co2(submit_script)
            pal, maxcore = parse_resource_settings(control_content)
            submit_script = apply_resources_to_submit_script(submit_script, pal, maxcore)
            
            submit_path = job_dir / "submit_job.sh"
            submit_path.write_text(submit_script)
            submit_path.chmod(0o755)
            
            # Job einreichen
            result = subprocess.run(
                ['sbatch', 'submit_job.sh'],
                cwd=job_dir,
                capture_output=True,
                text=True
            )
            
            if result.returncode == 0:
                job_id = result.stdout.strip().split()[-1]
                print(f"CO2 Job successfully submitted!")
                print(f"Job ID: {job_id}")
                print(f"Job Type: {job_type}")
                print(f"Directory: {job_dir}")
                print(f"")
                print(f"Files created:")
                print(f"  - CONTROL.txt")
                print(f"  - input.xyz ({num_atoms} atoms)")
                print(f"  - co2.xyz (3 atoms)")
                print(f"")
                print(f"Check status: squeue -u $USER")
                print(f"View output: tail -f {job_dir}/delfin_*.out")
                
                reset_form()
            else:
                print(f"Error submitting job:")
                print(result.stderr)
        
        except Exception as e:
            print(f"Error creating job: {str(e)}")

submit_button.on_click(handle_submit)

spacer = widgets.Label(value="", layout=widgets.Layout(height='15px'))

# === Layout ===
ui_left = widgets.VBox([
    job_name_widget,
    spacer,
    job_type_widget,
    spacer,
    coords_help,
    coords_widget,
    spacer,
    co2_info,
    spacer,
    control_help,
    control_widget,
    spacer,
    submit_button,
    output_area,
], layout=widgets.Layout(width='55%', padding='10px'))

ui_right = widgets.VBox([
    widgets.Label("Molecule visualization:"),
    mol_output,
], layout=widgets.Layout(width='45%', padding='10px'))

full_ui = widgets.HBox([ui_left, ui_right])
display(full_ui)

## Job-Status

In [ ]:
!squeue -u $USER

In [ ]:
!squeue -u $USER --start

In [ ]:
!ls -lht ~/calc_co2/

In [ ]:
# === Job Canceling ===
import ipywidgets as widgets
from IPython.display import display, clear_output
import subprocess

job_cancel_title = widgets.HTML("<h3>Job-Canceling</h3>")
job_cancel_help = widgets.Label("Enter a SLURM Job ID to cancel it.")
job_cancel_id = widgets.Text(
    value='',
    placeholder='e.g. 123456',
    description='Job ID:',
    layout=widgets.Layout(width='300px'),
    style={'description_width': 'initial'}
)
job_cancel_button = widgets.Button(
    description='CANCEL JOB',
    button_style='danger',
    layout=widgets.Layout(width='150px', height='40px')
)
job_cancel_output = widgets.Output()

def handle_job_cancel(button):
    with job_cancel_output:
        clear_output()
        job_id = job_cancel_id.value.strip()
        if not job_id.isdigit():
            print("Error: Please enter a numeric Job ID.")
            return
        result = subprocess.run(
            ['scancel', job_id],
            capture_output=True,
            text=True
        )
        if result.returncode == 0:
            print(f"Job {job_id} canceled.")
            job_cancel_id.value = ''
        else:
            print("Error canceling job:")
            print(result.stderr or result.stdout)

job_cancel_button.on_click(handle_job_cancel)

job_cancel_section = widgets.VBox([
    job_cancel_title,
    job_cancel_help,
    widgets.HBox([job_cancel_id, job_cancel_button]),
    job_cancel_output
], layout=widgets.Layout(padding='10px', border='1px solid #ddd'))

display(job_cancel_section)